In [1]:
import h2o
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as pca

### h2o INIT

In [2]:
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/felipe/miniconda3/envs/lab/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsgzxetuy
  JVM stdout: /tmp/tmpsgzxetuy/h2o_felipe_started_from_python.out
  JVM stderr: /tmp/tmpsgzxetuy/h2o_felipe_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_felipe_mh48qe
H2O cluster total nodes:,1
H2O cluster free memory:,8 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### LOAD DATA

In [3]:
titanic= h2o.import_file('titanic.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
titanic.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,nan,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,nan,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,nan,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,nan,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C


In [5]:
y_columns = "Survived"
x_columns = ["Pclass","Sex","Fare", "Embarked"]

In [6]:
titanic[x_columns]

Pclass,Sex,Fare,Embarked
3,male,7.25,S
1,female,71.2833,C
3,female,7.925,S
1,female,53.1,S
3,male,8.05,S
3,male,8.4583,Q
1,male,51.8625,S
3,male,21.075,S
3,female,11.1333,S
2,female,30.0708,C


### Categorical to numeric

In [7]:
titanic[y_columns] = titanic[y_columns].asfactor()
titanic[y_columns].levels()

[['0', '1']]

### Train and test Split

In [8]:
train, test=titanic.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

### PCA
#### As its own object

In [9]:
pca_model = pca(k = 3, transform = "STANDARDIZE", pca_method="Power",
                   use_all_factor_levels=True, impute_missing=True)
pca_model.train(training_frame=X_train)

pca Model Build progress: |███████████████████████████████████████████████| 100%


#### As attribute of the data 

In [10]:
X_train.pca=pca(k = 3, transform = "STANDARDIZE", pca_method="Power",
                   use_all_factor_levels=True, impute_missing=True)
X_train.pca.train(training_frame=X_train)

pca Model Build progress: |███████████████████████████████████████████████| 100%


In [11]:
X_train.pca.varimp(use_pandas=True)

,,pc1,pc2,pc3
0,Standard deviation,1.272174,1.094934,0.676130
1,Proportion of Variance,0.404322,0.299510,0.114208
2,Cumulative Proportion,0.404322,0.703832,0.818039


In [12]:
X_train.pca.rotation()


Rotation: 


,,pc1,pc2,pc3
0,Embarked.C,-0.089517,0.123905,0.052748
1,Embarked.Q,0.043921,0.017507,0.071316
2,Embarked.S,0.192444,0.702656,0.053075
3,Sex.female,-0.058890,0.233115,0.297470
4,Sex.male,0.205737,0.610953,-0.120330
5,Pclass,0.667733,-0.198685,0.695667
6,Fare,-0.679235,0.153405,0.634346


### Use Predict to apply to new data

In [13]:
pca_model.predict(X_test)

pca prediction progress: |████████████████████████████████████████████████| 100%


PC1,PC2,PC3
1.20859,1.31323,0.444239
-0.898956,0.723421,-0.72371
-1.02537,0.69487,-0.60565
-0.837496,0.456213,-0.489982
-0.422782,1.32548,0.570352
-0.880378,0.727617,-0.74106
-0.874348,0.0102864,-0.626226
0.692443,1.63426,1.09694
3.94363,2.36854,-1.93939
-1.2894,1.07284,-0.188405


In [14]:
X_train.pca.predict(X_test)

pca prediction progress: |████████████████████████████████████████████████| 100%


PC1,PC2,PC3
-1.20859,1.31323,-0.444239
0.898957,0.723421,0.72371
1.02537,0.69487,0.605649
0.837496,0.456213,0.489982
0.422782,1.32548,-0.570352
0.880378,0.727617,0.74106
0.874348,0.0102864,0.626226
-0.692443,1.63426,-1.09694
-3.94363,2.36854,1.93939
1.2894,1.07284,0.188405


### Random Forest Estimator

In [15]:
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

In [16]:
rf_fit1 = rf(model_id='rf_fit1', seed=1)
rf_fit1.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
rf_fit2 = rf(model_id='rf_fit2', ntrees=500, seed=1)
rf_fit2.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [18]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [19]:
print(rf_perf1.auc())
print(rf_perf2.auc())

0.8690939771547248
0.8727933541017653


In [20]:
rf_fit1.model_performance

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,68629.0,12.0,20.0,15.74,65.0,122.0,104.56




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14875914993519257
RMSE: 0.3856930773752526
LogLoss: 0.6696648778300591
Mean Per-Class Error: 0.20687112451818335
AUC: 0.8287330316742082
pr_auc: 0.7019398718787383
Gini: 0.6574660633484164

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5164389038085937: 


,,0,1,Error,Rate
0,0,390.0,52.0,0.1176,(52.0/442.0)
1,1,80.0,190.0,0.2963,(80.0/270.0)
2,Total,470.0,242.0,0.1854,(132.0/712.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-12-09 10:50:21,0.020 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2019-12-09 10:50:21,0.230 sec,1.0,0.438454,5.106405,0.767948,0.288372,2.126643,0.204918
2,,2019-12-09 10:50:21,0.325 sec,2.0,0.425639,4.164358,0.787466,0.407391,2.142593,0.212411
3,,2019-12-09 10:50:21,0.371 sec,3.0,0.426485,3.847543,0.791867,0.407914,2.098866,0.228240
4,,2019-12-09 10:50:21,0.410 sec,4.0,0.402409,3.218888,0.817833,0.428276,2.177370,0.204738
5,,2019-12-09 10:50:21,0.440 sec,5.0,0.406285,3.276443,0.814424,0.415450,2.109630,0.206897
6,,2019-12-09 10:50:21,0.479 sec,6.0,0.402841,2.968158,0.817340,0.431553,2.205109,0.209337
7,,2019-12-09 10:50:21,0.508 sec,7.0,0.399438,2.743483,0.821588,0.433517,2.279473,0.205539
8,,2019-12-09 10:50:21,0.539 sec,8.0,0.398831,2.592611,0.820573,0.453709,2.322528,0.207246
9,,2019-12-09 10:50:21,0.561 sec,9.0,0.390597,2.278357,0.827472,0.466643,2.408832,0.204871



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Sex,1966.183960,1.000000,0.432198
1,Fare,1826.943604,0.929182,0.401591
2,Pclass,579.268188,0.294615,0.127332
3,Embarked,176.869492,0.089956,0.038879


<bound method ModelBase.model_performance of >

In [21]:
rf_fit2.model_performance

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit2


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,500.0,500.0,668044.0,10.0,20.0,15.492,40.0,131.0,101.58




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14402656811864584
RMSE: 0.3795083241756969
LogLoss: 0.4823035871696077
Mean Per-Class Error: 0.20193564605329306
AUC: 0.8350259762024468
pr_auc: 0.800699593853175
Gini: 0.6700519524048936

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5245452530082615: 


,,0,1,Error,Rate
0,0,396.0,46.0,0.1041,(46.0/442.0)
1,1,81.0,189.0,0.3,(81.0/270.0)
2,Total,477.0,235.0,0.1784,(127.0/712.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-12-09 10:50:23,0.002 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2019-12-09 10:50:23,0.022 sec,1.0,0.438454,5.106405,0.767948,0.288372,2.126643,0.204918
2,,2019-12-09 10:50:23,0.037 sec,2.0,0.425639,4.164358,0.787466,0.407391,2.142593,0.212411
3,,2019-12-09 10:50:23,0.055 sec,3.0,0.426485,3.847543,0.791867,0.407914,2.098866,0.228240
4,,2019-12-09 10:50:23,0.074 sec,4.0,0.402409,3.218888,0.817833,0.428276,2.177370,0.204738
5,,2019-12-09 10:50:23,0.099 sec,5.0,0.406285,3.276443,0.814424,0.415450,2.109630,0.206897
6,,2019-12-09 10:50:23,0.120 sec,6.0,0.402841,2.968158,0.817340,0.431553,2.205109,0.209337
7,,2019-12-09 10:50:23,0.141 sec,7.0,0.399438,2.743483,0.821588,0.433517,2.279473,0.205539
8,,2019-12-09 10:50:23,0.159 sec,8.0,0.398831,2.592611,0.820573,0.453709,2.322528,0.207246
9,,2019-12-09 10:50:23,0.176 sec,9.0,0.390597,2.278357,0.827472,0.466643,2.408832,0.204871



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Fare,18537.431641,1.000000,0.417782
1,Sex,18253.474609,0.984682,0.411382
2,Pclass,5790.680664,0.312378,0.130506
3,Embarked,1789.524048,0.096536,0.040331


<bound method ModelBase.model_performance of >

In [22]:
rf_fit3 = rf(model_id='rf_fit3', seed=1, nfolds=5, ntrees=2000)
rf_fit3.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
rf_perf3 = rf_fit3.model_performance(test)
print(rf_perf3.auc())

0.8733125649013499


In [24]:
rf_fit3.model_performance

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit3


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,2000.0,2000.0,2636581.0,10.0,20.0,15.48,36.0,131.0,100.1205




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14269002192077124
RMSE: 0.3777433280956412
LogLoss: 0.47186758080864716
Mean Per-Class Error: 0.19967320261437904
AUC: 0.8384657281716105
pr_auc: 0.8104900487569271
Gini: 0.6769314563432209

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5844795274626371: 


,,0,1,Error,Rate
0,0,403.0,39.0,0.0882,(39.0/442.0)
1,1,84.0,186.0,0.3111,(84.0/270.0)
2,Total,487.0,225.0,0.1728,(123.0/712.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-12-09 10:50:59,29.325 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2019-12-09 10:50:59,29.332 sec,1.0,0.438454,5.106405,0.767948,0.288372,2.126643,0.204918
2,,2019-12-09 10:50:59,29.337 sec,2.0,0.425639,4.164358,0.787466,0.407391,2.142593,0.212411
3,,2019-12-09 10:50:59,29.341 sec,3.0,0.426485,3.847543,0.791867,0.407914,2.098866,0.228240
4,,2019-12-09 10:50:59,29.345 sec,4.0,0.402409,3.218888,0.817833,0.428276,2.177370,0.204738
5,,2019-12-09 10:50:59,29.349 sec,5.0,0.406285,3.276443,0.814424,0.415450,2.109630,0.206897
6,,2019-12-09 10:50:59,29.356 sec,6.0,0.402841,2.968158,0.817340,0.431553,2.205109,0.209337
7,,2019-12-09 10:50:59,29.360 sec,7.0,0.399438,2.743483,0.821588,0.433517,2.279473,0.205539
8,,2019-12-09 10:50:59,29.363 sec,8.0,0.398831,2.592611,0.820573,0.453709,2.322528,0.207246
9,,2019-12-09 10:50:59,29.370 sec,9.0,0.390597,2.278357,0.827472,0.466643,2.408832,0.204871



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Sex,74590.585938,1.000000,0.420750
1,Fare,73081.054688,0.979762,0.412235
2,Pclass,22443.039062,0.300883,0.126596
3,Embarked,7165.459473,0.096064,0.040419


<bound method ModelBase.model_performance of >

### Predict

In [25]:
y_pred1 = rf_fit1.predict(X_test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [26]:
y_pred1

predict,p0,p1
1,0.0089375,0.991062
1,0.0334055,0.966595
1,0.458584,0.541416
0,0.976032,0.0239683
0,0.861735,0.138265
0,0.873405,0.126595
0,0.621027,0.378973
0,0.551189,0.448811
0,0.821189,0.178811
0,0.991834,0.00816603


# autoML

In [34]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_20191209_105108_model_2,0.980392,0.250417,0.0735294,0.272504,0.0742583
XRT_1_AutoML_20191209_105108,0.966503,0.285959,0.0735294,0.298599,0.0891614
DRF_1_AutoML_20191209_105108,0.961601,0.280782,0.0874183,0.294499,0.0867298
StackedEnsemble_AllModels_AutoML_20191209_105108,0.937092,0.386763,0.131536,0.344611,0.118757
StackedEnsemble_BestOfFamily_AutoML_20191209_105108,0.934641,0.397556,0.131536,0.350178,0.122624
XGBoost_grid_1_AutoML_20191209_105108_model_1,0.919935,0.40139,0.158497,0.358677,0.12865
GBM_1_AutoML_20191209_105108,0.911356,0.383874,0.131536,0.343529,0.118012
GBM_4_AutoML_20191209_105108,0.895425,0.412947,0.146242,0.361616,0.130766
GBM_3_AutoML_20191209_105108,0.89134,0.420524,0.160131,0.364992,0.13322
GBM_2_AutoML_20191209_105108,0.888072,0.427274,0.172386,0.368701,0.13594


In [37]:
pred_automl = aml_ti.leader.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [47]:
pred_automl

predict,p0,p1
1,0.0349731,0.965027
1,0.0612363,0.938764
1,0.508873,0.491127
0,0.960478,0.0395219
0,0.856251,0.143749
0,0.927165,0.0728346
1,0.0669857,0.933014
1,0.513078,0.486922
0,0.903001,0.0969988
0,0.963528,0.0364717


Links:


https://medium.com/analytics-vidhya/gentle-introduction-to-automl-from-h2o-ai-a42b393b4ba2


https://www.kaggle.com/cooleel/a-quick-try-of-h2o-automl-on-titanic-dataset